# Load API Key and OPENAI Package

Loads all of the packages that will be used and the OpenAI API Key that will be used. Store your OpenAI API Key in a .env file.
If you are missing any of these packages run the following commands
```
$   pip3 install PyPDF2
$   pip3 install python.dotenv
$   pip3 install openai
```

In [4]:
import os
import openai
import PyPDF2
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
openai.organization = os.getenv("ORGANIZATION_KEY")
openai.api_key = os.getenv("OPENAI_API_KEY")

In [6]:
# Sample API Call
# response = openai.Completion.create(
#     engine="text-davinci-002",  # Specify the engine (model) you want to use
#     prompt="Translate the following English text to French: 'Hello, how are you?'",
#     max_tokens=50,  # Limit the length of the generated text
# )

# Retrieve Lesson Proposals

Lesson Proposals will be extracted using the PyPDF2 Package. You must have the lessons downloaded locally for this code to work and place it inside of a 
Lesson_Proposal folder. It will extract all of the text and put it into the <i style="color: red"><b>proposal_text</b></i> variable.

In [8]:
# Stores the path of All the PDFs in the Lesson_Proposals Folder
pdf_file_paths = []

proposal_list = os.listdir("./Lesson_Proposals/")

# Extracts all the PDF Paths
for i in proposal_list:
    pdf_file_paths.append(f"Lesson_Proposals/{i}")

In [9]:
# EXTRACT TEXT FROM EACH PROPOSAL

lesson_proposal = []

for i in pdf_file_paths:
    pdf_file = open(i, 'rb')
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    text = ''
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text += page.extract_text()
    
    pdf_file.close()
    lesson_proposal.append(text)


# Prompting GPT


OpenAI's API currently doesn't support a continuous chat as far as im concerned. There are a lot of repositories that address this problem but I think the solution I will be going with is a long continuous chain, starting with providing a rubric to score similiarities, providing an output format, then ultimately providing all of the proposals. We can test this with the online version GPT prior to using the API to see if it generates the result we want.

## Creating the Prompt

### Opening a Chat Log

In [ ]:
# class Chat:

### Providing a Rubric

You can find the Rubric that was used to score similarity [here](https://docs.google.com/document/d/1x18mVubT2H4Gj_GvDM3nUupCqYQHaH8Qk8UleAQjPyU/edit).

In [ ]:
# Telling GPT to use this Rubric when Grading the Proposals

rubric_message = """

"""

### Telling GPT How to Output the Results

In [ ]:
format_message = """
Please respond in the following format: 

1. Proposal 1 Title
    Most Similar: (Proposal Most Similar to)
        3-5 Sentences of Context comparing the 2 proposals
        Rubric Score:
            ...
    Least Similar: (Proposal Least Similar to)
        3-5 Sentences of Content comparing the 2 proposals
        Rubric Score:
            ...
2. Proposal 2 Title
    Same format as above
3. The same format until all proposals have been considered
"""


### Combining all of the messages

We will take all of the different messages created above and combine them together into one singular prompt that we can send to GPT.
It will HOPEFULLY output an appropriate response.

In [ ]:
prompt = f"""
You are going to receive several lesson proposals and your job is to compare them to find proposals that are most similar to each other. 
You will have a total of 7 proposals, A, B, C, etc. You will compare the proposals and see which is the most similar and least similar to each individual 
proposal. Each proposal will go through an individual comparison against each other. You will use the rubric that will be provided as a way to score
the similarities between each proposal. This score will be used in your response.

The Rubric:
{rubric_message}

{format_message}

Here are the following proposals

Proposal 1:
{lesson_proposal[0]}

Proposal 2:
{lesson_proposal[1]}

Proposal 3:
{lesson_proposal[2]}

Proposal 4:
{lesson_proposal[3]}

Proposal 5:
{lesson_proposal[4]}

Proposal 6:
{lesson_proposal[5]}

Proposal 7:
{lesson_proposal[6]}

"""

## Sending the Prompt

### Sending API Call

In [ ]:
# Using OPENAI's API to prompt gpt-4 to analyze the text
response = openai.Completion.create(
    engine="gpt-4",  # Specify the engine (model) you want to use
    prompt="Translate the following English text to French: 'Hello, how are you?'",
    max_tokens=5000,  # Limit the length of the generated text
)

### Extracting the Message

The following code will create a new file/overwrite the "GPT_Analysis.txt" with GPT's text analysis of the lesson proposals.

In [ ]:
# print(response['choices'][0]['text'])
GPT_Analysis = open("GPT_Analysis.txt", 'w')
GPT_Analysis.write(response['choices'][0]['text'])
GPT_Analysis.close()